This function will read the files with Mathemaica Pythonized expressions and save them to the arrays of strings

In [8]:
def import_expressions(file):
    file_expr=open(file,"r")
    list_expr=[line.rstrip() for line in file_expr]
    file_expr.close()
    return list_expr

In [9]:
eomsT_expr=import_expressions("EOMsT_py.txt")

eomsT_expr[1];

In [16]:
len(eomsT_expr)

9

In [ ]:
def evaluate_expressions(list_expr):
    l=len(list_expr)
    vals_expr=np.zeros((ne,nx,1),dtype=float)
    for i,expr in enumerate(listStringExpr):
        eoms[i]=eval(expr)

In [75]:
Expressions_files=("EOMsI_py.txt","EOMsT_py.txt","EOMsB_py.txt","CoefsI_py.txt","CoefsT_py.txt","CoefsB_py.txt")

In [83]:
eval?

In [85]:
globals();

In [164]:
class PDE_2d_problem:
    import numpy as np
    from numpy import sqrt, cosh, pi, sinh, tanh, exp, cos
    
    def __init__(self,  files, grid_vars, phys_vars={'c1':9.9, 'p0':1.2, 'mu1':1., 'mu':2, 'a0':0.3, 'nperiods':1}):
        from numpy import sqrt, cosh, pi, sinh, tanh, exp, cos
        self._eomsI_expr=import_expressions(files[0])
        self._eomsT_expr=import_expressions(files[1])
        self._eomsB_expr=import_expressions(files[2])
        self._coefsI_expr=import_expressions(files[3])
        self._coefsT_expr=import_expressions(files[4])
        self._coefsB_expr=import_expressions(files[5])
        self.nx, self.ny=grid_vars
        self.phys_vars=phys_vars
        self.ne=len(self._eomsI_expr)
        
        self.gy=np.reshape(np.random.rand(self.ny),(1,self.ny))
        self.gx=np.reshape(np.random.rand(self.nx),(self.nx,1))
        
    def EOMs(self,gfm):
        gft=np.reshape(gfm[:,:,:,-1],(self.ne,7,self.nx,1))
        gfb=np.reshape(gfm[:,:,:,0],(self.ne,7,self.nx,1))
        gfi=gfm[:,:,:,1:-1]
        gy=self.gy
        gx=self.gx
        gxi=gx
        gxt=gx
        gxb=gx
        gyi=gy[:,1:-1]
        
        r=eval(self._eomsI_expr[1],globals().update(locals()),self.phys_vars)
        return(r)
    

In [165]:
problem=PDE_2d_problem(Expressions_files,(4,5))

In [168]:
problem.EOMs(np.random.rand(9,7,4,5))

array([[ 0.32716626, -1.69250076, -0.51180528],
       [ 1.47639424,  0.10634241, -0.73239093],
       [ 3.66780251,  1.74113581, -8.00079951],
       [ 1.99135571,  1.45794345, -2.32932854]])

In [39]:
a,b=4,6

In [40]:
b

6